# Repair functions for Notebook DB setup

Hosted deployment in 20J required a particular setup for Mongo and Postgres databases. This notebook undoes those.

## Postgres

The updates for postgres notebooks were as follows:

___

## Setting up

The next group of cells set up your database connection, and reset the database to a clean state. Check notebook *08.1 Data Definition Language in SQL* if you are unsure what the next cells do.

You may need to change the given values of the variables `DB_USER` and `DB_PWD`, depending on which environment you are using

---

## MongoDB

---

### Setting your database credentials

In order to work with a database, we need to create a *connection* to the database. A connection allows us to manipulate the database, and query its contents (depending on what usage rights you have been granted). For the SQL notebooks in TM351, the details of your connection will depend upon whether you are using the OU-hosted server, accessed via [tm351.open.ac.uk](https:tm351.open.ac.uk), or whether you are using a version hosted on your own computer, which you should have set up using either Vagrant or Docker.

To set up the connection, you need a login name and a pasword. we will use the variables `DB_USER` and `DB_PWD` to hold the user name and password respectively that you will use to connect to the database. Run the appropriate cell to set your credentials in the following cells.

#### Connecting to the database on [tm351.open.ac.uk](https:tm351.open.ac.uk)

If you are using the Open University hosted server, you should execute the following cell, using your OUCU as the value of `DB_USER`, and the password you were given at the beginning of the module. Note that if the cell is in RAW NBconvert style, you will need to change its type to Code in order to execute it.

The variables `DB_USER` and `DB_PWD` are strings, and so you need to put them in quotes.

In this case, note that the connection string contains an additional option at the end: `?authsource=user-data`. For the MongoDB setup that we are using here, this option tells Mongo where to look for the authentication database.

#### Connecting to the database on a locally hosted machine

If you are running the Jupyter server on your own machine, via Docker or Vagrant, you should execute the following cell. Note that if the cell is in RAW NBconvert style, you will need to change its type to Code in order to execute it.

Note that the locally hosted versions of the environment give you full administrator rights, which is why you do not need to specify a user name or password. Obviously, this would not generally not be granted on a multi-user database, unless you are the database administrator.

---

## Generic

Code for managing patches

In [1]:
#%pdb

In [35]:
import nbformat
from pathlib import Path

def fix_cells(cell_type, str_start, path='.',
              replace_match=None, replace_with=None,
              convert_to=None, overwrite=True,
              version=nbformat.NO_CONVERT,
              ignore_files = None,
              verbose=False):
    """Remove cells of a particular type starting with a particular string.
       Optionally replace cell contents.
       Optionally convert cell type.
    """

    # Cell types
    cell_types = ['markdown', 'code', 'raw']
    if cell_type and cell_type not in cell_types:
        raise ValueError('Error: cell_type not recognised')
        
    if convert_to and convert_to not in cell_types:
        raise ValueError('Error: convert_to cell type not recognised')

    # Iterate path
    nb_dir = Path(path)
    for p in nb_dir.rglob("*"): #nb_dir.iterdir():
        if ignore_files and p.name in ignore_files:
            continue
        if '.ipynb_checkpoints' in p.parts:
            continue
        
        if p.is_file() and p.suffix == '.ipynb':
            updated = False
            if verbose:
                print(f"Checking {p}")

            # Read notebook
            with p.open('r') as f:
                # parse notebook
                #nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)
                #nb = nbformat.convert(nb, version)
                #opinionated
                try:
                    nb = nbformat.read(f, as_version=version)
                except:
                    print(f"Failed to open: {p}")
                    continue
                deletion_list = []
                for i, cell in enumerate(nb['cells']):
                    if cell["cell_type"]==cell_type and nb['cells'][i]["source"].startswith(str_start):
                        if replace_with is None and not convert_to:
                            deletion_list.append(i)
                        elif replace_with is not None:
                            if replace_match:
                                nb['cells'][i]["source"] = nb['cells'][i]["source"].replace(replace_match, replace_with)
                                updated = True
                            else:
                                nb['cells'][i]["source"] = replace_with
                                updated = True
                        if convert_to:
                            if convert_to=='code':
                                new_cell = nbformat.v4.new_code_cell(nb['cells'][i]["source"])
                                nb['cells'][i] = new_cell
                            elif convert_to=='markdown':
                                new_cell = nbformat.v4.new_markdown_cell(nb['cells'][i]["source"])
                                nb['cells'][i] = new_cell
                            elif convert_to=='raw':
                                new_cell = nbformat.v4.new_raw_cell(nb['cells'][i]["source"])
                                nb['cells'][i] = new_cell           
                            else:
                                pass
                            updated = True

                # Delete unrequired cells
                if deletion_list:
                    updated = True
                nb['cells']  = [c for i, c in enumerate(nb['cells']) if i not in deletion_list]

                if updated:
                    # Validate - exception if we fail
                    #nbformat.validate(nb)

                    # Create output filename
                    out_path =  p if overwrite else p.with_name(f'{p.stem}__patched{p.suffix}') 

                    # Save notebook
                    print(f"Updating: {p}")
                    nbformat.write(nb, out_path.open('w'), nbformat.NO_CONVERT)

## Postgres cleaner

In [36]:
ignore_files=['21J DB repair.ipynb']

str_start = '# If you are using the remote environment, change this cell'
fix_cells('raw', str_start, ignore_files=ignore_files)

Checking TM351_remote_VCE_test.ipynb
Checking Using Jupyter Notebooks - READ ME FIRST.ipynb
Checking grepping-notebooks.ipynb
Checking Part 01 Notebooks/01.1 Getting started with IPython and Jupyter Notebooks - Bootcamp.ipynb
Checking Part 01 Notebooks/01.2 Python recap.ipynb
Checking Part 01 Notebooks/01.3 Basic python data structures.ipynb
Checking Part 01 Notebooks/01.4 Defining new functions in python.ipynb
Checking Part 01 Notebooks/01.5 Python file handling.ipynb
Checking Part 01 Notebooks/01.X Customising the Notebook Environment.ipynb
Checking Part 02 Notebooks/02.1 Pandas Dataframes.ipynb
Checking Part 02 Notebooks/02.2  Data file formats.ipynb
Checking Part 02 Notebooks/02.2.0 Data file formats - file encodings.ipynb
Checking Part 02 Notebooks/02.2.1  Data file formats - CSV.ipynb
Checking Part 02 Notebooks/02.2.2  Data file formats - JSON.ipynb
Checking Part 02 Notebooks/02.2.3  Data file formats - other.ipynb
Checking Part 03 Notebooks/03.1 Cleaning data.ipynb
Checking Part

Checking Part 22 Notebooks/22.3 Applying the classifier to a real dataset.ipynb
Checking Part 22 Notebooks/22.4 Term frequency and inverse document frequency.ipynb
Checking Part 23 Notebooks/23.1  SQL injection hacks.ipynb
Checking Part 23 Notebooks/form_server.ipynb
Checking Part 23 Notebooks/form_server_safe.ipynb
Checking Part 23 Notebooks/part_23_authentication_notebook.ipynb
Checking Part 23 Notebooks/reset_form_server.ipynb
Checking Part 25 Notebooks/25.1 Exploring the RDFLib package.ipynb
Checking Part 25 Notebooks/25.2 Querying using SPARQL.ipynb
Checking Part 25 Notebooks/25.3 Endpoints - accessing real data.ipynb
Checking Part 26 Notebooks/26.1 Using multiple endpoints.ipynb
Checking Part 26 Notebooks/26.2 The SPARQL CONSTRUCT query and inferencing.ipynb
Checking Part 26 Notebooks/26.3 Visualisation.ipynb


In [ ]:
str_start = "# If you are using a locally hosted environment, change this cell"
replace_match ="""# If you are using a locally hosted environment, change this cell
# type to "code", and execute it

"""
replace_with = ''
convert_to = 'code'
fix_cells('raw', str_start, convert_to='code',
          replace_match=replace_match, replace_with=replace_with, ignore_files=ignore_files)

*Minor other changes noticed in notebooks 3.3 and 3.4 preview of git diffs and addressed manually.*